# Youtube Dataset Analysis

### Data Decription 

Our raw dataset contains information about 200 trending videos on YouTube across 205 days. This data was collected from Canada, United States, Great Britain, Germany, France, Russia, Mexico, South Korea, Japan, and India. It includes the video id, the date(s) the video was trending, the video title, the channel name, the number of likes, dislikes, and views at that trending date, the category ID corresponding to the relevant genre of the video, the comment count, the time the video was published, all its tags it was posted under, a link to the thumbnail image used for the video, a boolean values for whether or not the video ever had its comments disabled, ratings disabled, or had an error/was removed, a direct copy and paste of each video’s description, and the link ID to the video. Youtube determines if a video is trending by looking at various factors including number of views, shares, comments, and likes. This dataset was created to explore which videos are trending in certain countries and why, in regards to its raw data, perhaps to better understand how exactly a posted video goes viral and stays viral, and why it garners such attention. “Mitchell J”, a kaggle user, funded the creation of this dataset, and says in the description it was collected using YouTube’s own API’s to gather the relevant raw data. In that regard, since YouTube’s own API was used to scrape this data, only readily-available surface data on the video is included here. Essentially any statistic on a video that you can see when you watch a video is what is included here. 

In terms of pre-processing the data, we had to combine the files for all of the countries we decided to use into one data file. We decided to use only countries that had mostly characters that could be interpreted on the computer. These countries included Canada, Germany, France, Great Britain, and the United States. We decided to just look at the videos under the music category because Youtube is largely used for music. After loading all of the csv files and json files for each country, we determined the category number for music by looking at the json. We then filtered out any videos in each of the Canada, Germany, France, Great Britain, and the United States datasets that were not of the “Music” category. There were many repeats of trending videos on certain days across countries, so we added a column to the dataset that contained a list of countries that a particular video was trending in on a particular day. This allowed for us to just be able to append all of the datasets together and drop the duplicates to give us unique rows, but still distinguish which videos were trending when and in what countries. Additionally we dropped the columns with the link for the thumbnail image of the video, the category id, since they were all the same, and for the description of the video because we felt we would not use it in our analysis.

After this initial filtering, we still noticed some issues. Many videos had titles that were written in languages that did not port well into alpha-numeric characters, and showed up as glitchy unicode when looking at the raw data. In order to cut down on this, we filtered down each video that was essentially unusable due to its title/tags data. Each corresponding video per tag that did not contain only alphanumeric characters was flagged and removed from the csv. We also noticed many issues with the dating format of the videos. Each video had a corresponding column for the day it was trending, and the date it was published. The trending_day column was formatted oddly, in year/day/month, and the publish_date was a combined date/time string, so we formatted them all to be consistent. The publish_date was split into 2 columns, one representing the day, and one representing the exact time the video was uploaded. The trending_date column was formatted to the more standard month/day/year format, as was the published_day column. The publish_time column was left in 24 hour format to eliminate any issues with AM/PM. 

We combined multiple individual csv files, that were over 50 MB each, and json files into one csv file that is about 4 MB. After processing the raw data to combine multiple files into one cohesive dataset, our final dataset that we will use for this project contains trending videos, under the music category, over 205 days in Canada, Germany, France, Great Britain, and the United States. The attributes it contains are the video ids, the date the video was trending, the title of the video, the title of the channel of the video, the time the video was published, the date the video was published, the tags, the number of view, number of likes, number of dislikes, number of comments, and a list of the countries that a video was trending in on a particular day. 


Link to raw data: https://drive.google.com/drive/folders/1MEz1kqZ3AQVY_bdPxITEpsYqLlYEUBU-?usp=sharing

### Potential Problems
* Because we filtered out rows that contained unknown characters, we may find some gaps in our data. 
* An overall trend we pull may be incomplete as some foreign videos are missing
* For videos that are trending for more than one day, it may be difficult to reference them because their video id will be the same for both days. 
* We removed the “video description” column for all videos, which may impact our analysis


In [2]:
import pandas as pd

In [3]:
## Import Dataset
df = pd.read_csv("filtered_US_data.csv")
df.head()

,video_id,trending_date,title,channel_title,publish_time,tags,views,likes,dislikes,comment_count,countries_trending_in
0,5E4ZBSInqUU,17.14.11,Marshmello - Blocks (Official Music Video),marshmello,2017-11-13T17:00:00.000Z,"['marshmello', 'blocks', 'marshmello blocks', ...",687582,114188,1333,8371,['US']
1,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,2017-11-10T17:00:03.000Z,"['Eminem', 'Walk', 'On', 'Water', 'Aftermath/S...",17158531,787419,43420,125882,"['US', 'CA', 'GB', 'FR', 'DE']"
2,e_7zHm7GsYc,17.14.11,Hunter Hayes - You Should Be Loved (Part One O...,Hunter Hayes,2017-11-13T15:01:18.000Z,"['Hunter', 'Hayes', 'you should be loved', 'th...",13917,1318,24,76,['US']
3,zZ9FciUx6gs,17.14.11,Nickelback - The Betrayal Act III [Official Vi...,Nickelback,2017-11-13T15:31:44.000Z,"['Nickelback', 'Feed The Machine', 'The Betray...",57169,6927,161,565,['US']
4,PaJCFHXcWmM,17.14.11,U2 - The Blackout,U2VEVO,2017-11-13T17:00:04.000Z,"['U2', 'The', 'Blackout', 'Island', 'Records',...",60506,5389,106,455,['US']
